In [ ]:
python3 -c 'from pyalink.alink.download_pyalink_dep_jars import main;main()'

In [1]:
from pyalink.alink import *
resetEnv()
#useLocalEnv(1, config=None) 本地运行
useRemoteEnv("192.168.10.102", 8081, 2) #集群运行


Use one of the following commands to start using PyAlink:
 - useLocalEnv(parallelism, flinkHome=None, config=None): run PyAlink scripts locally.
 - useRemoteEnv(host, port, parallelism, flinkHome=None, localIp="localhost", config=None): run PyAlink scripts on a Flink cluster.
 - getMLEnv(): run PyAlink scripts as PyFlink scripts, support 'flink run -py xxx.py'.
Call resetEnv() to reset environment and switch to another.

JVM listening on 127.0.0.1:60932


MLEnv(benv=<pyflink.dataset.execution_environment.ExecutionEnvironment object at 0x0000025E1084EDF0>, btenv=<pyflink.table.table_environment.BatchTableEnvironment object at 0x0000025E1084EEE0>, senv=<pyflink.datastream.stream_execution_environment.StreamExecutionEnvironment object at 0x0000025E1084EBB0>, stenv=<pyflink.table.table_environment.StreamTableEnvironment object at 0x0000025E10873580>)

In [2]:
## prepare data
import numpy as np
import pandas as pd
data = np.array([
    [0, 0.0, 0.0, 0.0],
    [1, 0.1, 0.1, 0.1],
    [2, 0.2, 0.2, 0.2],
    [3, 9, 9, 9],
    [4, 9.1, 9.1, 9.1],
    [5, 9.2, 9.2, 9.2]
])
df = pd.DataFrame({"id": data[:, 0], "f0": data[:, 1], "f1": data[:, 2], "f2": data[:, 3]})
inOp = BatchOperator.fromDataframe(df, schemaStr='id double, f0 double, f1 double, f2 double')
FEATURE_COLS = ["f0", "f1", "f2"]
VECTOR_COL = "vec"
PRED_COL = "pred"

In [3]:
vectorAssembler = (
    VectorAssembler()
    .setSelectedCols(FEATURE_COLS)
    .setOutputCol(VECTOR_COL)
)

In [4]:
kMeans = (
    KMeans()
    .setVectorCol(VECTOR_COL)
    .setK(2)
    .setPredictionCol(PRED_COL)
)

In [5]:
pipeline = Pipeline().add(vectorAssembler).add(kMeans)
pipeline.fit(inOp).transform(inOp).firstN(9).collectToDataframe()

,id,f0,f1,f2,vec,pred
0,0.0,0.0,0.0,0.0,0.0 0.0 0.0,1
1,1.0,0.1,0.1,0.1,0.1 0.1 0.1,1
2,2.0,0.2,0.2,0.2,0.2 0.2 0.2,1
3,3.0,9.0,9.0,9.0,9.0 9.0 9.0,0
4,4.0,9.1,9.1,9.1,9.1 9.1 9.1,0
5,5.0,9.2,9.2,9.2,9.2 9.2 9.2,0
